In [1]:
from numba import jit
import numpy as np
import awkward.forth
import awkward as ak

In [9]:
machine = awkward.forth.ForthMachine32("10 20 if 999 then 123 if 987 then") 
machine.run()
machine.stack

[10, 999, 987]

In [10]:
bytecodes = ak.Array(
    ak.contents.ListOffsetArray(
        ak.index.Index64(machine.bytecodes_offsets),
    ak.contents.NumpyArray(machine.bytecodes),
    )
)
bytecodes

<Array [[0, 10, 0, 20, 3, 72, 0, 123, 3, 73], ...] type='3 * var * int64'>

In [104]:
def compile(bytecodes):
    pycode = ["@jit(nopython=True, nogil=True)\n", "def func():\n", "    cur = np.uint64(0)\n", "    stack = np.zeros(100, dtype=np.int64)\n"]
    for code in bytecodes:
        cursor = 0
        while cursor < len(code):
            if code[cursor] == 0:
                cursor += 1
                pycode.append(f"    cur = stack_push(stack, {code[cursor]}, cur)\n")
                cursor += 1
            elif code[cursor] == 38:
                pycode.append(f"    temp, cur = stack_pop(stack, cur)\n")
                cursor += 1
            elif code[cursor] == 44:
                pycode.append(f"    temp1,cur = stack_pop(stack, cur)\n")
                pycode.append(f"    temp2, cur = stack_pop(stack, cur)\n")
                pycode.append(f"    cur = stack_push(stack, temp1+temp2, cur)\n")
                cursor += 1
            elif code[cursor] == 45:
                pycode.append(f"    temp1,cur = stack_pop(stack, cur)\n")
                pycode.append(f"    temp2, cur = stack_pop(stack, cur)\n")
                pycode.append(f"    cur = stack_push(stack, temp2-temp1, cur)\n")
                cursor += 1
    return pycode

In [105]:
pycode = compile(ak.to_list(bytecodes))
pycode.append("    print(stack, cur)\n")
pycode.append("func()")
aa = "".join(pycode)

In [106]:
@jit(nopython=True, nogil=True)
def stack_push(stack,elem, cur):
    if cur == len(stack)-1:
        raise ValueError("Index out of range")
    stack[cur] = elem
    cur += 1
    return cur

@jit(nopython=True, nogil=True)
def stack_pop(stack, cur):
    if cur == 0:
        raise ValueError("Index out of range")
    cur -= 1
    return stack[cur], cur


In [107]:
print(aa)

@jit(nopython=True, nogil=True)
def func():
    cur = np.uint64(0)
    stack = np.zeros(100, dtype=np.int64)
    cur = stack_push(stack, 10, cur)
    temp, cur = stack_pop(stack, cur)
    cur = stack_push(stack, 10, cur)
    cur = stack_push(stack, 10, cur)
    temp1,cur = stack_pop(stack, cur)
    temp2, cur = stack_pop(stack, cur)
    cur = stack_push(stack, temp1+temp2, cur)
    cur = stack_push(stack, 30, cur)
    temp1,cur = stack_pop(stack, cur)
    temp2, cur = stack_pop(stack, cur)
    cur = stack_push(stack, temp2-temp1, cur)
    print(stack, cur)
func()


In [108]:
exec(aa)

[-10  30   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0] 1


In [49]:
aa

'@jit(nopython=True, nogil=True)\ndef func():\ncur = np.uint64(0)\nstack = np.zeros(100, dtype=np.int64)\n    cur = stack.push(stack, 10, cur)\n    temp, cur = stack.pop(stack, cur)\\]\n    cur = stack.push(stack, 10, cur)\n    cur = stack.push(stack, 10, cur)\n    temp1,cur = stack.pop(stack, cur)\n    temp2, cur = stack.pop(stack, cur)\n    cur = stack.push(stack, temp1+temp2, cur)\n'